In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install selenium
!pip install scrapy

     |████████████████████████████████| 958 kB 9.2 MB/s 
     |████████████████████████████████| 356 kB 42.0 MB/s 
     |████████████████████████████████| 138 kB 52.6 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 3.6 MB 53.3 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 254 kB 7.5 MB/s 
     |████████████████████████████████| 251 kB 54.8 MB/s 

In [3]:

import requests
import scrapy
from scrapy import Selector

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import openpyxl
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import re
import math 

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [9]:
def get_airline_list(A_Z_LIST):
  res = requests.get(A_Z_LIST)
  soup2 = BeautifulSoup(res.text, 'html.parser')
  airlines = []
  for list in soup2.select("div.a_z_col_group > ul.items > li > a"):
    airline = list.get_text().lower().strip().replace(" ","-")
    airlines.append(airline)

  return airlines

In [10]:
def lastPage( airline_name):
  plusUrl = airline_name
  url = baseUrl + urllib.parse.quote_plus(airline_name)
  req = requests.get(url)
  data = BeautifulSoup(req.text, 'html.parser')
  reviewCount = int(re.findall("\d+", data.find(attrs={'itemprop':'reviewCount'}).text)[0])
  lastPage = math.ceil(reviewCount/10)
  return lastPage

In [11]:
def get_airline_reviews(airline):

    basic_final = list()
    title_final = list()
    date_final = list()
    content_final = list()
    airline_final = list()
    rate_final = list()
    SeatType_all = list()
    AircraftType_all = list()
    SeatLayout_all = list()
    PageNum = 1
    lastpage = lastPage(airline)

    while PageNum < lastpage + 1:

      res = requests.get(
          f"{baseUrl}{airline}/page/{PageNum}/")
      soup = BeautifulSoup(res.text, 'html.parser')

      # rate
      all = soup.find_all("div",{"class":"rating-10"})
      for x in all:
        try:
          rate_final.append(x.find("span", {"itemprop":"ratingValue"}).get_text())
        except:
          rate_final.append('NA')
      rate_final = [i for i in rate_final if len(i)<=2 or i=='NA']
    

      # airline, title, content, date
      review = soup.select("div.body")
      #print(review)
      for data in review:
        airline_final.append(airline)
        title = data.select_one('h2').get_text()
        title_final.append(title)
        date = data.select_one('time').get_text()
        date_final.append(date)
        content = data.select_one('div.text_content').get_text()
        content_final.append(content)
        #print(data)
        table = data.find_all("tr")
        #print(table)
        SeatType = table[0].find_all("td")[1].get_text()
        SeatType_all.append(SeatType)
        AircraftType = table[1].find_all("td")[1].get_text()
        AircraftType_all.append(AircraftType)
        SeatLayout = table[2].find_all("td")[1].get_text()
        SeatLayout_all.append(SeatLayout)
      
        #type = soup.select_one('td:contains("Type Of Traveller") + td').text
      
      PageNum += 1

    table_all = pd.DataFrame({'airline':airline_final, 'title':title_final, 'date':date_final, 'content':content_final, 'rate':rate_final,
                          'SeatType':SeatType_all,'AircraftType':AircraftType_all, 'SeatLayout':SeatLayout_all })
    return table_all

In [15]:
def get_seat_reviews(AQ_seat_reviews):
    not_working = ['atlasglobal','azul-airlines','air-canada-rouge','boliviana-de-aviacion','hk-express','indigo',
                    'middle-east-airlines','pakistan-intl-airlines','sas-scandinavian','sata-international',
                    'small-planet-airlines','swiss-intl-air-lines','ukraine-international','vietjet-air',
                    'virgin-atlantic','westjet-airlines']  ##their urls are not same to airlinename
    i = 0
    for al in airlines[i:]:
      i+=1
      if al not in not_working:
        #print(al)
        AQ_seat_reviews = pd.concat([AQ_seat_reviews,get_airline_reviews(al)])
        print(al,'\t',AQ_seat_reviews.shape)
        
    return AQ_seat_reviews

In [16]:
#def __init__(self,airline_reviews= False,seat_reviews = True,OUTPUT_FILE_SEAT = '/content/drive/MyDrive/Colab Notebooks/Hackathon/AQS_data_seat.csv',OUTPUT_FILE_AIRLINES = '/content/drive/MyDrive/Colab Notebooks/Hackathon/AQA_data_airline.csv'):
# self.airline_reviews = airline_reviews
# self.seat_reviews = seat_reviews
OUTPUT_FILE_SEAT = '/content/drive/MyDrive/Colab Notebooks/Hackathon/AQS_data_seat.csv'
#OUTPUT_FILE_AIRLINES = OUTPUT_FILE_AIRLINES

#if self.seat_reviews:
baseUrl = 'https://www.airlinequality.com/seat-reviews/'
A_Z_LIST = 'https://www.airlinequality.com/review-pages/a-z-seat-reviews/'

airlines = get_airline_list(A_Z_LIST)
columns_list = ['airline', 'title', 'date', 'content', 'rate']
AQ_seat_reviews = pd.DataFrame(columns=columns_list)
AQ_seat_reviews = get_seat_reviews(AQ_seat_reviews)


aegean-airlines 	 (14, 8)
aer-lingus 	 (22, 8)
aeroflot-russian-airlines 	 (42, 8)
aerolineas-argentinas 	 (43, 8)
aeromexico 	 (50, 8)
air-astana 	 (52, 8)
air-berlin 	 (75, 8)
air-busan 	 (76, 8)
air-canada 	 (149, 8)
air-china 	 (164, 8)
air-corsica 	 (166, 8)
air-dolomiti 	 (167, 8)
air-europa 	 (168, 8)
air-france 	 (276, 8)
air-india 	 (287, 8)
air-india-express 	 (288, 8)
air-italy 	 (289, 8)
air-mauritius 	 (295, 8)
air-new-zealand 	 (369, 8)
air-north-yukon's-airline 	 (370, 8)
air-serbia 	 (373, 8)
air-tahiti-nui 	 (380, 8)
air-transat 	 (395, 8)
airasia 	 (406, 8)
airasia-india 	 (408, 8)
airasia-philippines 	 (410, 8)
airasia-x 	 (413, 8)
alaska-airlines 	 (420, 8)
alitalia 	 (445, 8)
allegiant-air 	 (453, 8)
american-airlines 	 (558, 8)
american-eagle 	 (561, 8)
ana-all-nippon-airways 	 (586, 8)
asiana-airlines 	 (615, 8)
austrian-airlines 	 (637, 8)
avianca 	 (639, 8)
ba-cityflyer 	 (640, 8)
bamboo-airways 	 (642, 8)
bangkok-airways 	 (645, 8)
batik-air 	 (646, 8)
blue-ai

In [ ]:
AQ_seat_reviews.to_csv('/content/drive/MyDrive/Colab Notebooks/Hackathon/AQS.csv')